In [74]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from collections import defaultdict

df = pd.read_csv(r"Z:/SEM 5/20XW58 MACHINE LEARNING LAB/ws6-Naive-bayes-classifier/datasets/wbc_original.csv")
del df['id']

print(df)

     clump_thickness  size_uniformity  shape_uniformity  marginal_adhesion  \
0                  5                1                 1                  1   
1                  5                4                 4                  5   
2                  3                1                 1                  1   
3                  6                8                 8                  1   
4                  4                1                 1                  3   
..               ...              ...               ...                ...   
694                3                1                 1                  1   
695                2                1                 1                  1   
696                5               10                10                  3   
697                4                8                 6                  4   
698                4                8                 8                  5   

     epithelial_size bare_nucleoli  bland_chromatin  normal_nuc

In [75]:
features = df.columns
features = list(features)
# print(features)
features = features[0:len(features)-1]
# print(features)
# print(len(features))

In [101]:
def remove_multicollinearity(features):    
    final_features = [x for x in features]
    CR = df[features].corr().values.tolist()

    for i in range(len(CR)):
        for j in range(i+1,len(CR)):
            if abs(CR[i][j])>0.7 and features[i] in final_features:
                final_features.remove(features[i])

    return final_features

final_features = remove_multicollinearity(features)
print(final_features)

['clump_thickness', 'marginal_adhesion', 'epithelial_size', 'bare_nucleoli', 'bland_chromatin', 'normal_nucleoli', 'mitoses']


In [102]:
features_list = []
for i in range(0,len(final_features)):
    features_list.append(np.array(df[final_features[i]]))

# print(y)
# print(features_list)
y = df['class']
for i in range(len(y)):
    if y[i]==2:
        y[i]=0
    elif y[i]==4:
        y[i]=1

final_features.remove('bare_nucleoli') #dataset char problem
x_data = df[final_features]
y_data = pd.DataFrame(y)

In [166]:
def def_value():
    return 0

def makeCPT(x,y):
    CPT_YES = defaultdict(def_value)
    count = 0
    for k in range(1,11):
        for j in range(1,2):        
            for i in range(len(x)):
                if(y[i]==1 and x[i][j]==k):
                    count += 1
            CPT_YES[k] = count/np.count_nonzero(y == 1)
#     np.count_nonzero(x == 2)
    print(CPT_YES)
    
    pass

In [167]:
def naiveBayesClassifier(x,y):
    no_of_features = len(x[0])
#     identifying unique quantities
    unique_quantities = [set()]*6
    
    for j in range(no_of_features):
        for i in range(len(x)):
            unique_quantities[j].add(x[i][j])
#     print(unique_quantities)
    print(y)
    makeCPT(x,y)
    pass

In [143]:
X_train,X_test,Y_train,Y_test = train_test_split(x_data.to_numpy(),y_data.to_numpy(), test_size=0.33)
# print(X_train,Y_train)


In [168]:
naiveBayesClassifier(X_train,Y_train)

[[0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
